In [2]:
# API_KEY = "sk-proj-hwpvzXEvm6w-2nU-**-Zw6dNemnYv32_iloRSEQsJNM7EjjiVoCGkj7szMq4Jlxx4x_sfQlEA"

In [3]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding


llm_model = OpenAI(
    model="gpt-4o-mini",
    api_key=API_KEY
)

embed_model = OpenAIEmbedding(
    model="text-embedding-ada-002",
    api_key=API_KEY
)


Settings.llm = llm_model
Settings.embed_model = embed_model

In [4]:
import requests

def mensaje_por_telegram(mensaje):
    """Útil cuando se solicita enviar un MENSAJE a Telegram"""

    url_getIdChat = 'https://api.telegram.org/bot'+bot_token+'/getUpdates'
    response = requests.get(url_getIdChat)

    bot_message = mensaje
    url_sendMessage = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatId + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(url_sendMessage)
    return response.text

In [5]:
mensaje_por_telegram("hola, funciona el mensaje?")

'{"ok":true,"result":{"message_id":257,"from":{"id":1060305894,"is_bot":true,"first_name":"\\ud83c\\udfc1script","username":"edustratiobot"},"chat":{"id":125821294,"first_name":"Eduardo","last_name":"FC","type":"private"},"date":1751701634,"text":"hola, funciona el mensaje?"}}'

# Tool

In [6]:
from llama_index.core.tools import FunctionTool

telegram_tool = FunctionTool.from_defaults(
    mensaje_por_telegram,
    description="Útil cuando se solicita enviar un MENSAJE a Telegram"
)

# ReAct Agent

In [7]:
from llama_index.core.agent import ReActAgent

agent = ReActAgent.from_tools(
    [telegram_tool], verbose=True
)

In [8]:
user_query = "enviame un mensaje por telegram que diga: INCREIBLE QUE UN LLM SEA CAPAZ DE USAR LA API DE TELGRAM"

In [9]:
response = agent.chat(user_query)
response

> Running step 13de4939-5881-4e8f-9781-0a5c539c085a. Step input: enviame un mensaje por telegram que diga: INCREIBLE QUE UN LLM SEA CAPAZ DE USAR LA API DE TELGRAM
Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: mensaje_por_telegram
Action Input: {'mensaje': 'INCREIBLE QUE UN LLM SEA CAPAZ DE USAR LA API DE TELGRAM'}
Observation: {"ok":true,"result":{"message_id":258,"from":{"id":1060305894,"is_bot":true,"first_name":"\ud83c\udfc1script","username":"edustratiobot"},"chat":{"id":125821294,"first_name":"Eduardo","last_name":"FC","type":"private"},"date":1751701955,"text":"INCREIBLE QUE UN LLM SEA CAPAZ DE USAR LA API DE TELGRAM"}}
> Running step a2b31624-5535-45ce-ae8e-082b412a84b7. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: El mensaje ha sido enviado correctamente por Telegram.


AgentChatResponse(response='El mensaje ha sido enviado correctamente por Telegram.', sources=[ToolOutput(content='{"ok":true,"result":{"message_id":258,"from":{"id":1060305894,"is_bot":true,"first_name":"\\ud83c\\udfc1script","username":"edustratiobot"},"chat":{"id":125821294,"first_name":"Eduardo","last_name":"FC","type":"private"},"date":1751701955,"text":"INCREIBLE QUE UN LLM SEA CAPAZ DE USAR LA API DE TELGRAM"}}', tool_name='mensaje_por_telegram', raw_input={'args': (), 'kwargs': {'mensaje': 'INCREIBLE QUE UN LLM SEA CAPAZ DE USAR LA API DE TELGRAM'}}, raw_output='{"ok":true,"result":{"message_id":258,"from":{"id":1060305894,"is_bot":true,"first_name":"\\ud83c\\udfc1script","username":"edustratiobot"},"chat":{"id":125821294,"first_name":"Eduardo","last_name":"FC","type":"private"},"date":1751701955,"text":"INCREIBLE QUE UN LLM SEA CAPAZ DE USAR LA API DE TELGRAM"}}', is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

In [13]:
print(agent.get_prompts()['agent_worker:system_prompt'].template)

You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

NEVER surround your response with markdown code markers. You may use code markers within your response if y

In [16]:
# Podríamos usar más tools?

from llama_index.core import StorageContext, load_index_from_storage

sc = StorageContext.from_defaults(persist_dir="vector_store")
index = load_index_from_storage(sc)
query_engine = index.as_query_engine()

In [18]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

rag_tool = QueryEngineTool(
    query_engine=query_engine,
    metadata=ToolMetadata(
        name="Seguros",
        description="Útil cuando hay que dar respuesta a preguntas relacionadas con seguros de Hogar y coberturas"
    )
)

In [19]:
agent2 = ReActAgent.from_tools(
    [telegram_tool, rag_tool], verbose=True
)

In [20]:
user_query = "tengo alguna garantía que cubra los humos?"

response = agent2.chat(user_query)

> Running step de0c9f88-8dbb-45dd-b2bb-72d540a67a5d. Step input: tengo alguna garantía que cubra los humos?
Thought: El usuario está preguntando sobre garantías que cubran los humos, lo que sugiere que se refiere a seguros de hogar. Necesito usar la herramienta adecuada para obtener información sobre esto.
Action: Seguros
Action Input: {'input': '¿tengo alguna garantía que cubra los humos?'}
Observation: Sí, hay garantías que cubren los daños ocasionados por humo. Estas garantías protegen tanto la vivienda como los bienes contenidos en ella, incluyendo los daños causados por humo que pueden resultar de incendios, explosiones o incluso incidentes menores, como una llamarada en la cocina. Esto significa que si el humo daña elementos como la campana extractora o el techo, los costos de reparación y restauración estarían cubiertos.
> Running step c9716304-853b-4d21-9917-eb74c6eba533. Step input: None
Thought: Puedo responder sin usar más herramientas. Utilizaré el idioma del usuario para r

AgentChatResponse(response='Sí, hay garantías que cubren los daños ocasionados por humo. Estas garantías protegen tanto la vivienda como los bienes contenidos en ella, incluyendo los daños causados por humo que pueden resultar de incendios, explosiones o incluso incidentes menores, como una llamarada en la cocina. Esto significa que si el humo daña elementos como la campana extractora o el techo, los costos de reparación y restauración estarían cubiertos.', sources=[ToolOutput(content='Sí, hay garantías que cubren los daños ocasionados por humo. Estas garantías protegen tanto la vivienda como los bienes contenidos en ella, incluyendo los daños causados por humo que pueden resultar de incendios, explosiones o incluso incidentes menores, como una llamarada en la cocina. Esto significa que si el humo daña elementos como la campana extractora o el techo, los costos de reparación y restauración estarían cubiertos.', tool_name='Seguros', raw_input={'input': '¿tengo alguna garantía que cubra 

In [21]:
response.response

'Sí, hay garantías que cubren los daños ocasionados por humo. Estas garantías protegen tanto la vivienda como los bienes contenidos en ella, incluyendo los daños causados por humo que pueden resultar de incendios, explosiones o incluso incidentes menores, como una llamarada en la cocina. Esto significa que si el humo daña elementos como la campana extractora o el techo, los costos de reparación y restauración estarían cubiertos.'

In [22]:
user_query = "enviame un mensaje por telegram que diga: INCREIBLE QUE UN LLM SEA CAPAZ DE USAR LA API DE TELGRAM"

response = agent2.chat(user_query)

> Running step 20420b90-8056-4879-8dd6-07393d7bd46d. Step input: enviame un mensaje por telegram que diga: INCREIBLE QUE UN LLM SEA CAPAZ DE USAR LA API DE TELGRAM
Thought: The current language of the user is: Spanish. I need to use a tool to help me answer the question.
Action: mensaje_por_telegram
Action Input: {'mensaje': 'INCREIBLE QUE UN LLM SEA CAPAZ DE USAR LA API DE TELGRAM'}
Observation: {"ok":true,"result":{"message_id":259,"from":{"id":1060305894,"is_bot":true,"first_name":"\ud83c\udfc1script","username":"edustratiobot"},"chat":{"id":125821294,"first_name":"Eduardo","last_name":"FC","type":"private"},"date":1751702609,"text":"INCREIBLE QUE UN LLM SEA CAPAZ DE USAR LA API DE TELGRAM"}}
> Running step 8157696a-fb83-4dc9-884c-19102aaf66d4. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: El mensaje ha sido enviado correctamente a Telegram.


In [23]:
response.response

'El mensaje ha sido enviado correctamente a Telegram.'

In [26]:
user_query = "si tengo alguna garantía que cubra humos entre mis coberturas de hogar, envíamelas por Telegram"

response = agent2.chat(user_query)

> Running step eee8c4ef-7afb-43f2-ba51-c80b042f675e. Step input: si tengo alguna garantía que cubra humos entre mis coberturas de hogar, envíamelas por Telegram
Thought: Necesito verificar si tienes alguna garantía que cubra humos en tus coberturas de hogar antes de enviar un mensaje por Telegram.
Action: Seguros
Action Input: {'input': '¿Tengo alguna garantía que cubra humos en mis coberturas de hogar?'}
Observation: Sí, hay garantías que cubren los daños ocasionados por humo en las coberturas de hogar. Estas garantías protegen contra los daños causados por humo debido a escapes repentinos en cocinas, sistemas de calefacción y otros aparatos eléctricos. Esto incluye la reparación de daños en la vivienda y sus contenidos, como por ejemplo, la reparación de una campana extractora dañada y la pintura del techo afectado por el humo.
> Running step f2d212d9-47b9-41e0-b5fa-5af9284d62c4. Step input: None
Thought: Puedo enviar la información sobre las garantías que cubren humos a Telegram.
Ac

In [27]:
response.response

'La información ha sido enviada correctamente. Si necesitas algo más, no dudes en preguntar.'

# OpenAI agent

In [28]:
from llama_index.agent.openai import OpenAIAgent

agent_openai = OpenAIAgent.from_tools(
    [telegram_tool], verbose=True
)

In [29]:
response = agent_openai.chat(user_query)
response

Added user message to memory: si tengo alguna garantía que cubra humos entre mis coberturas de hogar, envíamelas por Telegram
=== Calling Function ===
Calling function: mensaje_por_telegram with args: {"mensaje":"Para verificar si tienes alguna garantía que cubra humos entre tus coberturas de hogar, es necesario que consultes directamente con tu compañía de seguros o revises tu póliza. Las coberturas pueden variar según la aseguradora y el tipo de póliza que tengas."}
Got output: {"ok":true,"result":{"message_id":262,"from":{"id":1060305894,"is_bot":true,"first_name":"\ud83c\udfc1script","username":"edustratiobot"},"chat":{"id":125821294,"first_name":"Eduardo","last_name":"FC","type":"private"},"date":1751702987,"text":"Para verificar si tienes alguna garant\u00eda que cubra humos entre tus coberturas de hogar, es necesario que consultes directamente con tu compa\u00f1\u00eda de seguros o revises tu p\u00f3liza. Las coberturas pueden variar seg\u00fan la aseguradora y el tipo de p\u00f

AgentChatResponse(response='He enviado un mensaje a tu Telegram informándote que para verificar si tienes alguna garantía que cubra humos entre tus coberturas de hogar, es necesario que consultes directamente con tu compañía de seguros o revises tu póliza. Las coberturas pueden variar según la aseguradora y el tipo de póliza que tengas.', sources=[ToolOutput(content='{"ok":true,"result":{"message_id":262,"from":{"id":1060305894,"is_bot":true,"first_name":"\\ud83c\\udfc1script","username":"edustratiobot"},"chat":{"id":125821294,"first_name":"Eduardo","last_name":"FC","type":"private"},"date":1751702987,"text":"Para verificar si tienes alguna garant\\u00eda que cubra humos entre tus coberturas de hogar, es necesario que consultes directamente con tu compa\\u00f1\\u00eda de seguros o revises tu p\\u00f3liza. Las coberturas pueden variar seg\\u00fan la aseguradora y el tipo de p\\u00f3liza que tengas."}}', tool_name='mensaje_por_telegram', raw_input={'args': (), 'kwargs': {'mensaje': 'Para